In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils import Connection

In [2]:
with Connection() as connection:
    raw_patient_features_df = pd.read_sql('SELECT * FROM "TUKGRP2"."EX3_PATIENT_FEATURES"', connection)
raw_patient_features_df.head(3)

,AGE,GENDER,BMI,SYSTOLICBP,DIASTOLICBP,NISTcode
0,58,0,51.156317,124.049689,76.633540,3
1,57,0,26.508474,132.526315,76.526315,1
2,81,1,29.135000,159.432835,77.641791,3


In [9]:
patient_features_df = raw_patient_features_df.round(0)
patient_features_df = patient_features_df[patient_features_df['BMI'] <= 50]
patient_features_df = patient_features_df[patient_features_df['BMI'] >= 10]
patient_features_df = pd.get_dummies(patient_features_df, columns=['GENDER', 'NISTcode'])
patient_features_df.head(10)

,AGE,BMI,SYSTOLICBP,DIASTOLICBP,GENDER_0,GENDER_1,NISTcode_0,NISTcode_1,NISTcode_2,NISTcode_3,NISTcode_4,NISTcode_5,NISTcode_9
1,57,27.0,133.0,77.0,1,0,0,1,0,0,0,0,0
2,81,29.0,159.0,78.0,0,1,0,0,0,1,0,0,0
3,73,29.0,139.0,81.0,1,0,0,1,0,0,0,0,0
4,43,28.0,125.0,78.0,0,1,0,0,0,1,0,0,0
5,49,30.0,144.0,88.0,0,1,0,1,0,0,0,0,0
6,37,31.0,131.0,81.0,1,0,0,0,1,0,0,0,0
7,44,27.0,129.0,78.0,1,0,0,0,0,0,1,0,0
8,58,33.0,136.0,76.0,1,0,0,0,0,1,0,0,0
9,44,22.0,116.0,72.0,1,0,0,0,0,0,1,0,0
10,86,36.0,119.0,66.0,0,1,0,0,0,0,1,0,0


In [12]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import linear_model, metrics

def get_train_test(patient_features_df, bp_type):
    bp_columns = ['SYSTOLICBP', 'DIASTOLICBP']
    target = patient_features_df[bp_type]
    data = patient_features_df[patient_features_df.columns.difference(bp_columns)]
    return data, target

def train_test_regression(data, target):
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=1)
    regression = linear_model.LinearRegression()
    regression.fit(X_train, y_train)
    y_pred = regression.predict(X_test)
    print("R2: {}".format(metrics.r2_score(y_test, y_pred)))
    return regression

data_sys, target_sys = get_train_test(patient_features_df, 'SYSTOLICBP')
data_dia, target_dia = get_train_test(patient_features_df, 'DIASTOLICBP')

model_sys = train_test_regression(data_sys, target_sys)
model_dia = train_test_regression(data_dia, target_dia)


R2: 0.17991479416693545
R2: 0.11167192675383564
